In [129]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import warnings
warnings.filterwarnings('ignore')

In [149]:
#importing tracking data
tracking_data = pd.read_csv('tracking_data.csv')

In [27]:
#importing metadata
with open('metadata.json','r') as f:
    metadata = json.load(f)

In [99]:
#fullback runs
## left or right back
## start when players speed reached atleast 5m/s and acceleration 2.5 m/s2
## continue till speed is less than 4m/s and accceleration -2.5m/s2
### period_id,frameidx_start,frameidx_end,game_clock_start,game_clock_end,team_id,player_id,player_number,speed_start,speed_end,acceleration_start,acceleration_end
#overlapping runs
#space and how much exterior

In [340]:
def fullbackruns(tracking_data,i):
    df = tracking_data[(tracking_data['game_clock']>=0.08)&(tracking_data['player_id'] == i)].reset_index()
    df['acceleration'] = df['speed'].diff() / df['game_clock'].diff()
    df['acceleration'].fillna(0,inplace=True)
    fullbackruns_start_df= pd.DataFrame(columns=['team_id','player_id','player_number','period_id','frameidx_start','game_clock_start','speed_start','acceleration_start','x_start','y_start','ball_x_start','ball_y_start','possession_x_start','possession_y_start'])
    fullbackruns_end_df= pd.DataFrame(columns=['frameidx_end','game_clock_end','speed_end','acceleration_end','x_end','y_end','ball_x_end','ball_y_end','possession_x_end','possession_y_end'])
    row=0
    while (row < len(df)):
        if ((df.loc[row,'live'] == True) and df.loc[row,'team_id']==df.loc[row,'last_touch_team_id'] and (df.loc[row,'speed'] >= 5) and (df.loc[row,'acceleration'] >= 2.5)):
            df_poss = tracking_data[tracking_data['frame_idx'] == df.loc[row,'frame_idx']].reset_index()
            ball_x = df_poss.loc[0,'ball_x']
            ball_y = df_poss.loc[0,'ball_y']
            possession = df_poss[(ball_x-1.5 <= df_poss['x']) & (df_poss['x'] <= ball_x+1.5)&(ball_y-1.5 <= df_poss['y']) & (df_poss['y'] <= ball_y+1.5)]
            if len(possession) == 0:
                fullbackruns_start_df = fullbackruns_start_df.append({'team_id':df.loc[row,'team_id'],'player_id':df.loc[row,'player_id'],'player_number':df.loc[row,'player_number'],'period_id':df.loc[row,'period_id'],'frameidx_start':df.loc[row,'frame_idx'],'game_clock_start':df.loc[row,'game_clock'],'speed_start':df.loc[row,'speed'],'acceleration_start':df.loc[row,'acceleration'],'x_start':df.loc[row,'x'],'y_start':df.loc[row,'y'],'ball_x_start':df.loc[row,'ball_x'],'ball_y_start':df.loc[row,'ball_y'],'possession_x_start':ball_x,'possession_y_start':ball_y},ignore_index=True)
            else:
                fullbackruns_start_df = fullbackruns_start_df.append({'team_id':df.loc[row,'team_id'],'player_id':df.loc[row,'player_id'],'player_number':df.loc[row,'player_number'],'period_id':df.loc[row,'period_id'],'frameidx_start':df.loc[row,'frame_idx'],'game_clock_start':df.loc[row,'game_clock'],'speed_start':df.loc[row,'speed'],'acceleration_start':df.loc[row,'acceleration'],'x_start':df.loc[row,'x'],'y_start':df.loc[row,'y'],'ball_x_start':df.loc[row,'ball_x'],'ball_y_start':df.loc[row,'ball_y'],'possession_x_start':possession['x'].values[0],'possession_y_start':possession['y'].values[0]},ignore_index=True)
            row+=1
            print(row)
            while(row < len(df)):
                if (df.loc[row,'live']==False or df.loc[row,'team_id']!=df.loc[row,'last_touch_team_id'] or df.loc[row,'speed']<=4 or df.loc[row,'acceleration']<-2.5):
                    df_poss = tracking_data[tracking_data['frame_idx'] == df.loc[row,'frame_idx']].reset_index()
                    ball_x = df_poss.loc[0,'ball_x']
                    ball_y = df_poss.loc[0,'ball_y']
                    possession = df_poss[(ball_x-1.5 <= df_poss['x']) & (df_poss['x'] <= ball_x+1.5)&(ball_y-1.5 <= df_poss['y']) & (df_poss['y'] <= ball_y+1.5)]
                    if len(possession) == 0:
                        fullbackruns_end_df = fullbackruns_end_df.append({'frameidx_end':df.loc[row,'frame_idx'],'game_clock_end':df.loc[row,'game_clock'],'speed_end':df.loc[row,'speed'],'acceleration_end':df.loc[row,'acceleration'],'x_end':df.loc[row,'x'],'y_end':df.loc[row,'y'],'ball_x_end':df.loc[row,'ball_x'],'ball_y_end':df.loc[row,'ball_y'],'possession_x_end':ball_x,'possession_y_end':ball_y},ignore_index=True)
                    else:
                        fullbackruns_end_df = fullbackruns_end_df.append({'frameidx_end':df.loc[row,'frame_idx'],'game_clock_end':df.loc[row,'game_clock'],'speed_end':df.loc[row,'speed'],'acceleration_end':df.loc[row,'acceleration'],'x_end':df.loc[row,'x'],'y_end':df.loc[row,'y'],'ball_x_end':df.loc[row,'ball_x'],'ball_y_end':df.loc[row,'ball_y'],'possession_x_end':possession['x'].values[0],'possession_y_end':possession['y'].values[0]},ignore_index=True)
                    row+=1
                    break
                else:
                    row+=1
        else:
            row+=1
    fullbackruns_df_each = pd.concat([fullbackruns_start_df,fullbackruns_end_df],axis=1)
    return fullbackruns_df_each

In [349]:
player_ids = [443002,246099]
fullbackruns_df = pd.DataFrame()
for i in player_ids:
    fullbackruns_df_each = fullbackruns(tracking_data,i)
    fullbackruns_df = fullbackruns_df.append(fullbackruns_df_each,ignore_index=True)

154
7459
9090
11720
17061
23545
30824
33501
42571
51535
57685
58699
61476
63416
65848
66031
67389
67744
70115
70252
75809
79280
83777
87154
102048
109174
117830
121279
121400
124284
130652
140547
5311
5477
7149
8021
10248
11793
13336
14279
15367
15531
21752
24050
25635
28128
29247
37016
40971
42383
45651
47478
51722
51869
54300
57215
58664
61635
61971
69831
70277
72079
75848
76179
77377
77663
79348
79937
81441
85368
104180
106406
109126
109423
110645
111029


In [350]:
fullbackruns_df

,team_id,player_id,player_number,period_id,frameidx_start,game_clock_start,speed_start,acceleration_start,x_start,y_start,...,frameidx_end,game_clock_end,speed_end,acceleration_end,x_end,y_end,ball_x_end,ball_y_end,possession_x_end,possession_y_end
0,6900.0,443002.0,4.0,1.0,155.0,6.20,5.48,3.00,14.34,-24.57,...,168.0,6.72,5.36,-2.75,12.24,-26.53,21.28,0.01,21.47,0.18
1,6900.0,443002.0,4.0,1.0,7460.0,298.40,5.09,3.25,20.30,-25.28,...,7471.0,298.84,5.06,-3.00,19.59,-23.06,19.08,-22.41,19.59,-23.06
2,6900.0,443002.0,4.0,1.0,9091.0,363.64,5.24,2.75,0.80,-22.81,...,9197.0,367.88,5.91,-2.75,-26.93,-32.01,-28.21,-20.49,-27.00,-19.78
3,6900.0,443002.0,4.0,1.0,11721.0,468.84,5.19,2.75,0.23,-24.68,...,11768.0,470.72,5.58,-2.75,-10.37,-29.47,-5.40,-19.87,-5.40,-19.87
4,6900.0,443002.0,4.0,1.0,17062.0,682.48,5.25,3.00,-22.14,-27.97,...,17095.0,683.80,5.97,-2.75,-30.00,-27.40,-30.85,-19.73,-30.85,-19.73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,6900.0,246099.0,2.0,2.0,106409.0,1508.40,5.00,3.00,24.50,-30.77,...,106448.0,1509.96,7.79,3.25,33.86,-26.83,28.76,-22.11,28.96,-22.49
72,6900.0,246099.0,2.0,2.0,109129.0,1617.20,5.00,4.25,-44.34,-25.54,...,109368.0,1626.76,5.79,-2.75,21.90,-19.37,13.69,10.59,13.99,11.11
73,6900.0,246099.0,2.0,2.0,109426.0,1629.08,5.09,2.75,30.61,-20.09,...,109457.0,1630.32,6.46,-2.50,38.42,-21.21,37.81,-18.49,37.81,-18.49
74,6900.0,246099.0,2.0,2.0,110648.0,1677.96,5.03,3.25,-42.92,-4.61,...,110793.0,1683.76,7.76,-2.50,-8.83,-25.43,-6.70,-24.36,-6.70,-24.36


In [319]:
#extract the player ids (left or right back)from json data
#remove with less than one run maybe

In [360]:
for i in range(len(fullbackruns_df)):
    ball_mid_x = (fullbackruns_df.iloc[i]['ball_x_start']+fullbackruns_df.iloc[i]['ball_x_end']) / 2
    ball_mid_y = (fullbackruns_df.iloc[i]['ball_y_start']+fullbackruns_df.iloc[i]['ball_y_end']) / 2
    possession_mid_x = (fullbackruns_df.iloc[i]['possession_x_start']+fullbackruns_df.iloc[i]['possession_x_end']) / 2
    possession_mid_y = (fullbackruns_df.iloc[i]['possession_y_start']+fullbackruns_df.iloc[i]['possession_y_end']) / 2
    if (fullbackruns_df.iloc[i]['ball_y_start']>0 and fullbackruns_df.iloc[i]['ball_y_end']>0):
        if (fullbackruns_df.iloc[i]['x_start']<possession_mid_x<fullbackruns_df.iloc[i]['x_end'] or fullbackruns_df.iloc[i]['x_end']<possession_mid_x<fullbackruns_df.iloc[i]['x_start']) and (fullbackruns_df.iloc[i]['x_start']<ball_mid_x<fullbackruns_df.iloc[i]['x_end'] or fullbackruns_df.iloc[i]['x_end']<ball_mid_x<fullbackruns_df.iloc[i]['x_start'] and (fullbackruns_df.iloc[i]['y_end']>possession_mid_y and fullbackruns_df.iloc[i]['y_end']>possession_mid_y)):
            fullbackruns_df.loc[i,'overlap'] = 1
        else:
             fullbackruns_df.loc[i,'overlap'] = 0
    elif (fullbackruns_df.iloc[i]['ball_y_start']<0 and fullbackruns_df.iloc[i]['ball_y_end']<0):
        if (fullbackruns_df.iloc[i]['x_start']<possession_mid_x<fullbackruns_df.iloc[i]['x_end'] or fullbackruns_df.iloc[i]['x_end']<possession_mid_x<fullbackruns_df.iloc[i]['x_start']) and (fullbackruns_df.iloc[i]['x_start']<ball_mid_x<fullbackruns_df.iloc[i]['x_end'] or fullbackruns_df.iloc[i]['x_end']<ball_mid_x<fullbackruns_df.iloc[i]['x_start'] and (fullbackruns_df.iloc[i]['y_end']<possession_mid_y and fullbackruns_df.iloc[i]['y_end']<possession_mid_y)):
            fullbackruns_df.loc[i,'overlap'] = 1
        else:
             fullbackruns_df.loc[i,'overlap'] = 0

0
1
2
3
5
6
7
8
9
10
11
12
13
14
15
17
18
19
20
21
22
23
24
26
27
28
29
30
32
33
34
35
36
37
38
41
42
43
44
45
48
49
50
51
52
53
54
55
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
73
74
75


In [362]:
fullbackruns_df

,team_id,player_id,player_number,period_id,frameidx_start,game_clock_start,speed_start,acceleration_start,x_start,y_start,...,game_clock_end,speed_end,acceleration_end,x_end,y_end,ball_x_end,ball_y_end,possession_x_end,possession_y_end,overlap
0,6900.0,443002.0,4.0,1.0,155.0,6.20,5.48,3.00,14.34,-24.57,...,6.72,5.36,-2.75,12.24,-26.53,21.28,0.01,21.47,0.18,0.0
1,6900.0,443002.0,4.0,1.0,7460.0,298.40,5.09,3.25,20.30,-25.28,...,298.84,5.06,-3.00,19.59,-23.06,19.08,-22.41,19.59,-23.06,0.0
2,6900.0,443002.0,4.0,1.0,9091.0,363.64,5.24,2.75,0.80,-22.81,...,367.88,5.91,-2.75,-26.93,-32.01,-28.21,-20.49,-27.00,-19.78,1.0
3,6900.0,443002.0,4.0,1.0,11721.0,468.84,5.19,2.75,0.23,-24.68,...,470.72,5.58,-2.75,-10.37,-29.47,-5.40,-19.87,-5.40,-19.87,1.0
4,6900.0,443002.0,4.0,1.0,17062.0,682.48,5.25,3.00,-22.14,-27.97,...,683.80,5.97,-2.75,-30.00,-27.40,-30.85,-19.73,-30.85,-19.73,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,6900.0,246099.0,2.0,2.0,106409.0,1508.40,5.00,3.00,24.50,-30.77,...,1509.96,7.79,3.25,33.86,-26.83,28.76,-22.11,28.96,-22.49,1.0
72,6900.0,246099.0,2.0,2.0,109129.0,1617.20,5.00,4.25,-44.34,-25.54,...,1626.76,5.79,-2.75,21.90,-19.37,13.69,10.59,13.99,11.11,0.0
73,6900.0,246099.0,2.0,2.0,109426.0,1629.08,5.09,2.75,30.61,-20.09,...,1630.32,6.46,-2.50,38.42,-21.21,37.81,-18.49,37.81,-18.49,1.0
74,6900.0,246099.0,2.0,2.0,110648.0,1677.96,5.03,3.25,-42.92,-4.61,...,1683.76,7.76,-2.50,-8.83,-25.43,-6.70,-24.36,-6.70,-24.36,1.0


In [367]:
fullback_overlap = fullbackruns_df[fullbackruns_df['overlap'] == 1].reset_index(drop=True)

In [368]:
fullback_overlap

,team_id,player_id,player_number,period_id,frameidx_start,game_clock_start,speed_start,acceleration_start,x_start,y_start,...,game_clock_end,speed_end,acceleration_end,x_end,y_end,ball_x_end,ball_y_end,possession_x_end,possession_y_end,overlap
0,6900.0,443002.0,4.0,1.0,9091.0,363.64,5.24,2.75,0.80,-22.81,...,367.88,5.91,-2.75,-26.93,-32.01,-28.21,-20.49,-27.00,-19.78,1.0
1,6900.0,443002.0,4.0,1.0,11721.0,468.84,5.19,2.75,0.23,-24.68,...,470.72,5.58,-2.75,-10.37,-29.47,-5.40,-19.87,-5.40,-19.87,1.0
2,6900.0,443002.0,4.0,1.0,33502.0,1340.08,5.12,3.75,-16.62,-31.63,...,1341.36,6.03,-2.50,-24.20,-31.56,-22.27,-25.26,-22.27,-25.26,1.0
3,6900.0,443002.0,4.0,1.0,42572.0,1702.88,5.13,3.00,-22.82,-23.93,...,1706.60,5.98,-2.75,-39.77,-22.01,-35.87,-14.03,-35.29,-13.94,1.0
4,6900.0,443002.0,4.0,1.0,51536.0,2061.44,6.41,2.50,-14.16,-23.90,...,2063.44,6.51,-2.50,-27.00,-29.48,-28.10,-28.22,-27.00,-29.48,1.0
5,6900.0,443002.0,4.0,1.0,61477.0,2459.08,5.02,3.25,23.53,-31.96,...,2460.80,5.59,-2.50,13.47,-32.77,9.16,-27.67,9.16,-27.67,1.0
6,6900.0,443002.0,4.0,1.0,67745.0,2709.80,5.26,2.50,-17.22,-27.02,...,2710.32,4.85,-2.75,-14.62,-27.68,-13.81,-27.72,-14.62,-27.68,1.0
7,6900.0,443002.0,4.0,2.0,70118.0,56.76,5.17,4.75,24.98,22.59,...,57.88,6.20,-2.50,31.72,22.75,29.13,28.24,29.93,28.89,1.0
8,6900.0,443002.0,4.0,2.0,70255.0,62.24,5.00,3.25,40.23,27.05,...,63.80,6.31,-2.50,48.16,21.19,48.90,20.45,48.16,21.19,1.0
9,6900.0,443002.0,4.0,2.0,79283.0,423.36,5.03,2.75,-8.68,30.84,...,427.40,5.36,-2.50,14.64,25.15,15.18,24.96,14.64,25.15,1.0
